<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       ServiceNow Multipath Pattern Analysis
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>

<p style = 'font-size:18px;font-family:Arial'><b>Introduction</b></p>
<p style = 'font-size:16px;font-family:Arial'>ServiceNow is a widely-used platform for managing IT service requests and incidents. While it effectively captures case-level data and status transitions, it lacks native capabilities to perform deep-dive pattern analysis across the lifecycle of a case — especially in terms of how cases move through different support groups, how long they stay in each state, and what behaviors lead to resolutions or escalations. Due to limitations in ServiceNow's native analytics capabilities, answering questions like below are extremely difficult :<ul style = 'font-size:16px;font-family:Arial'>
    <li>How many cases moved from L1 to L2 multiple times?</li>
    <li>Which cases were resolved in a single touch by L2 teams?</li>
    <li>Which assignment groups handled the highest severity incidents before resolution?</li>
    </ul>
<p style = 'font-size:18px;font-family:Arial'><b>Business Value</b></p>
<p style = 'font-size:16px;font-family:Arial'>By integrating ServiceNow case audit data with Vantage's nPath® functionality we can unlock critical insights from the case history:<ul style = 'font-size:16px;font-family:Arial'>
    <li style = 'font-size:16px;font-family:Arial'><b>Operational Insights </b>like Understand common and rare resolution paths, identify escalation trends from L1 to L2 or other teams, discover patterns in delayed resolutions or reassignment loops</li>
    <li style = 'font-size:16px;font-family:Arial'><b>Improved Efficiency and SLA Management</b> like Highlight cases that violate SLAs due to excessive handoffs, spot 'one-touch' resolution patterns for knowledge reuse, enable proactive case routing strategies.</li>
    <li style = 'font-size:16px;font-family:Arial'><b>Enhanced Reporting and Decision-Making </b>like Weekly trend analysis for leadership using dashboards, support data-driven staffing decisions by understanding group-wise workloads.</li>
    </ul>
</p>
<p style = 'font-size:18px;font-family:Arial'><b>Why Vantage?</b></p>
<p style = 'font-size:16px;font-family:Arial'>Vantage has unique analytic capabilities for  looking for pattern. nPath® enables sequence mining and event pattern detection across logs or audit trails using syntax similar to regex, making it ideal for modeling complex case journeys. Teradata can handle large volumes of ServiceNow audit logs efficiently, enabling transformation of raw audit data into enriched analytic tables without compromising performance.</p> 


<hr style="height:2px;border:none;">

<p style = 'font-size:20px;font-family:Arial'><b>1. Connect to Vantage,  import python packages and explore the dataset</b></p>

<p style = 'font-size:16px;font-family:Arial'>Let us start with importing the required libraries, set environment variables and connect to Vantage.</p>

In [ ]:
#import libraries
import getpass
import warnings

from teradataml import *
import matplotlib.pyplot as plt


display.max_rows = 5

warnings.filterwarnings('ignore')

<p style = 'font-size:16px;font-family:Arial'>We will be prompted to provide the password. We will enter the password, press the Enter key, and then use the down arrow to go to the next cell. Begin running steps with Shift + Enter keys. </p>

In [ ]:
%run -i ../startup.ipynb
eng = create_context(host = 'host.docker.internal', username='demo_user', password = password)
print(eng)

In [ ]:
%%capture
execute_sql('''SET query_band='DEMO=FF_ServiceNow_Multipath_PatternAnalysis_Python.ipynb;' UPDATE FOR SESSION;''')

<p style = 'font-size:16px;font-family:Arial'>Begin running steps with Shift + Enter keys.</p>

<hr style="height:2px;border:none;">

<p style = 'font-size:20px;font-family:Arial'><b>2. Getting Data for This Demo</b></p>
<p style = 'font-size:16px;font-family:Arial'>We have provided data for this demo on cloud storage. We have the option of either running the demo using foreign tables to access the data without using any storage on our environment or downloading the data to local storage, which may yield somewhat faster execution. However, we need to consider available storage. There are two statements in the following cell, and one is commented out. We may switch which mode we choose by changing the comment string.
</p>

In [ ]:
%run -i ../run_procedure.py "call get_data('DEMO_ServiceNow_cloud');"
# takes about 20sec minute, estimated space: 0 MB
# %run -i ../run_procedure.py "call get_data('DEMO_ServiceNow_local');"
# takes about 1 minute, estimated space: 20 MB

<p style = 'font-size:16px;font-family:Arial'>Optional step – We should execute the below step only if we want to see the status of databases/tables created and space used.</p>

In [ ]:
%run -i ../run_procedure.py "call space_report();"

<hr style="height:2px;border:none;">
<p style = 'font-size:20px;font-family:Arial'><b>3. Analyze the raw data set</b></p>
<p style = 'font-size:16px;font-family:Arial'> Sys audit table in ServiceNow stores the assignment groups and states and all the updates that happened on the ticket. We have joined this data with other tables to build a raw data table. It's a mix of different tables (sys_audit, sys_user etc) in ServiceNow that allows us to identify what we think are the most important parts for that we need for the analysis of a journey.<br>One of the column of interest is the audit time stamp. This is the time stamp of when an event occurred. There are two very different dates in the table, the date of when a case was created and then the date of when something happened. We also have column for how long it has happened since the since the case or incident was created (hrs_elapsed). Table also captures the highest severity of the tickets. Let us take a look at the data in the table.</p> 

In [ ]:
tdf= DataFrame(in_schema('DEMO_ServiceNow', 'Audit_Raw_Data'))
tdf

<p style = 'font-size:16px;font-family:Arial'>We can check data for one of the tickets.</p>

In [ ]:
filtered_tdf = tdf[tdf['sn_ticket'] == 'INC0342089']
filtered_tdf.sort('sn_audit_TS')

<p style = 'font-size:16px;font-family:Arial'>If we look at the ticket data above we can see that the assignment group when the ticket was created was 'Cloud BaaS Operator' and later it was reassigned to 'GSO L1 Cloud Support'. There can be other tickets where this type of reassignment is done or there may be tickets where there is multiple reassignments. If we want to see the pattern or number of tickets where this happens we can easily do that using Vantage's nPath function.</p>
<p style = 'font-size:16px;font-family:Arial'>The nPath function scans a set of rows, looking for patterns that you specify. For each set of input rows that matches the pattern, nPath produces a single output row. The function provides a flexible pattern-matching capability that lets you specify complex patterns in the input data and define the values that are output for each matched input set.</p>

<p style = 'font-size:16px;font-family:Arial'>First let us check the values in the sn_ag column which holds the values for the assigment groups. </p>

In [ ]:
catsum = CategoricalSummary(data=tdf,
                             target_columns=['sn_ag']
                            )
 
catsum.result.sort('DistinctValue').head(20)

<p style = 'font-size:16px;font-family:Arial'>We have also observed that if there is any reassigmnet on the ticket the column fieldname = 'assignment_group' is set and the value is updated in the sn_ag column but if there is no reassigment of service group then this value is not set i.e the Audit_Raw_Data captures the changes on the ticket but not the initial assigment group. Hence we will first create a view by doing union on  all the initial values of assigment groups on the ticket and the subsequent reassignments; and then use this in our Npath query.</p>

In [ ]:
query='''
Replace view tickets_assignment_groups as
select 
    tablename,
    sn_ticket,
    sn_audit_converted AS sn_audit_ts,
    sn_audit_week_start_date,
    t1,
    hrs_elapsed,
    'assignment_group' AS fieldname,
    sn_ag AS fieldname_value
    FROM DEMO_ServiceNow.Audit_Raw_Data
    WHERE fieldname = 'assignment_group'
union
select 
    tablename,
    sn_ticket,
    sn_created_on AS sn_audit_ts,
    sn_audit_week_start_date,
    t1,
    hrs_elapsed,
    'assignment_group' AS fieldname,
    sn_ag AS fieldname_value
    FROM DEMO_ServiceNow.Audit_Raw_Data
    QUALIFY Row_number() OVER( PARTITION BY sn_ticket
        ORDER BY sn_audit_ts ASC) = 1
 ;'''
execute_sql(query)

In [ ]:
tdf2=DataFrame('tickets_assignment_groups')
tdf2

<p style = 'font-size:16px;font-family:Arial'>As a first example we want to build the metric <b>l2_cloudl3_bounces</b> that counts the number of times a ticket has been bounced between L2 and L3 using Teradata's <b>nPath</b> function.<br>
<ul style = 'font-size:16px;font-family:Arial'>We do this first defining two main symbols:
    <li>L2: This symbol represents the assignment groups that are considered L2 support groups.</li>
    <li>Cloud_L3: This symbol represents the assignment groups that are considered L3 support groups.</li>
    <li>OTHER: This symbol represents all rows as shown in the npath documentation.</li>
</ul>
<p style = 'font-size:16px;font-family:Arial'>
We then define our pattern as follows:
<br>The pattern <b>'(OTHER?.L2.Cloud_L3.OTHER?)+'</b>  means that we are looking for sequences where there is at least one occurrence of <b>L2</b>, followed by at least one occurrence of <b>Cloud_L3</b>, and then followed by any number of <b>OTHER</b> rows.<br>
        The <b>+</b> at the end indicates that this pattern can repeat one or more times.
<br>
Finally, we use the RESULT clause to extract the following information:    
<ul style = 'font-size:16px;font-family:Arial'>
<li><b>sn_ticket</b>: The ticket number.</li>
<li><b>first_l2_to_cloudl3_ts</b>: The timestamp of the first occurrence of the L2 to Cloud L3 transition.</li>
<li><b>sn_ags</b>: The total number of assignment groups involved in the ticket's journey.</li>
<li><b>l2_cloudl3_bounces</b>: The count of how many times the ticket bounced between L2 and Cloud L3.</li>
<li><b>path</b>: An accumulated list of assignment groups involved in the ticket's journey.</li>
<li><b>path_sn_audit_week</b>: An accumulated list of the week start dates for each assignment group in the ticket's journey.</li>
</ul></p>

In [ ]:
npath_l2_l3 = NPath(data1 = tdf2, 
                      data1_partition_column = ['sn_ticket'], 
                      data1_order_column = 'sn_audit_ts', 
                      mode = 'OVERLAPPING', 
                      symbols = ['fieldname = \'assignment_group\' AND fieldname_value IN (\'Cloud EU Azure System Operator\', \'Cloud EU BaaS Operator\', \'Cloud EU Ops Lead\', \'Cloud EU UPaaS System Operator\', \'Cloud EU VaaS Network Operator\',\'Cloud Ops AWS Super Users\',\'Cloud Antares AWS System Operator\',\'Cloud AWS Change Operator\',\'Cloud AWS System Operator\',\'Cloud AZURE Change Operator\',\'Cloud Azure System Operator\',\'Cloud BaaS Operator\',\'Cloud DRaaS Operator\',\'Cloud GCP Change Operator\',\'Cloud GCP System Operator\',\'Cloud UPaaS System Operator\',\'Cloud VaaS Deployment Operator\',\'Cloud VaaS Network Operator\') AS L2',
                                 'fieldname = \'assignment_group\' AND (fieldname_value IN (\'Cloud BaaS L3 Operator\', \'Cloud DRaaS L3 Operator\', \'Cloud L3 Security Applications Administrator\', \'Cloud Data Protection Operator\')) AS Cloud_L3',
                                 'TRUE as OTHER'],
                      pattern = '(OTHER?.L2.Cloud_L3.OTHER?)+', 
                      result = ['FIRST (sn_ticket of L2) as sn_ticket', 
                                'FIRST (sn_audit_ts of Cloud_L3) as first_l2_to_cloudl3_ts',
                                'COUNT (fieldname_value of ANY(L2,Cloud_L3) ) as sn_ags', 
                                'COUNT (fieldname_value of ANY(Cloud_L3) ) as l2_cloudl3_bounces',
                                'ACCUMULATE(10000,TRUE) (CAST(fieldname_value AS VARCHAR(20)) OF ANY(L2,Cloud_L3)) AS path,'
                                'ACCUMULATE(10000,TRUE) (CAST(sn_audit_week_start_date AS VARCHAR(100)) OF ANY(L2,Cloud_L3)) AS path_sn_audit_week'
                               ])
npath_l2_l3.result

<p style = 'font-size:16px;font-family:Arial'>For the distinct values we can use row_number() window function.
    

In [ ]:
d=npath_l2_l3.result
window=d.window(partition_columns="sn_ticket",order_columns="sn_ags",sort_ascending=False)
df1=window.row_number()
tdf_l2_l3=df1[(df1.col_row_number==1)].assign(drop_columns = True,
                                        sn_ticket=df1.sn_ticket,
                                        first_l2_to_cloudl3_ts=df1.first_l2_to_cloudl3_ts,
                                        sn_ags=df1.sn_ags,
                                        l2_cloudl3_bounces=df1.l2_cloudl3_bounces,
                                        path=df1.path,
                                        path_sn_audit_week=df1.path_sn_audit_week
                                       )
tdf_l2_l3

<p style = 'font-size:16px;font-family:Arial'>We can take another example where we want to find out the tickets that were touched by any L2 group.</p>

In [ ]:
npath_l2 = NPath(data1 = tdf2, 
                      data1_partition_column = ['sn_ticket'], 
                      data1_order_column = 'sn_audit_ts', 
                      mode = 'NONOVERLAPPING', 
                      symbols = ['fieldname = \'assignment_group\' AND fieldname_value IN (\'Cloud EU Azure System Operator\', \'Cloud EU BaaS Operator\', \'Cloud EU Ops Lead\', \'Cloud EU UPaaS System Operator\', \'Cloud EU VaaS Network Operator\',\'Cloud Ops AWS Super Users\',\'Cloud Antares AWS System Operator\',\'Cloud AWS Change Operator\',\'Cloud AWS System Operator\',\'Cloud AZURE Change Operator\',\'Cloud Azure System Operator\',\'Cloud BaaS Operator\',\'Cloud DRaaS Operator\',\'Cloud GCP Change Operator\',\'Cloud GCP System Oprator\',\'Cloud UPaaS System Operator\',\'Cloud VaaS Deployment Operator\',\'Cloud VaaS Network Operator\') AS L2',
                                 'TRUE as OTHER'],
                      pattern = 'L2*', 
                      result = ['FIRST (sn_ticket of L2) as sn_ticket', 
                                'FIRST (sn_audit_ts of L2) as first_l2_touched_ts',
                                'COUNT (fieldname_value of ANY(L2) ) as sn_ags', 
                                'ACCUMULATE(10000,TRUE) (CAST(fieldname_value AS VARCHAR(20)) OF ANY(L2)) AS path,'
                                'ACCUMULATE(10000,TRUE) (CAST(sn_audit_week_start_date AS VARCHAR(10)) OF ANY(L2)) AS l2_path_sn_audit_ts'
                               ])

d=npath_l2.result
window=d.window(partition_columns="sn_ticket",order_columns="sn_ags",sort_ascending=False)
df1=window.row_number()
tdf_l2=df1[(df1.col_row_number==1)].assign(drop_columns = True,
                                        sn_ticket=df1.sn_ticket,
                                        first_l2_touched_ts=df1.first_l2_touched_ts,
                                        sn_ags=df1.sn_ags,
                                        path=df1.path,
                                        l2_path_sn_audit_ts=df1.l2_path_sn_audit_ts
                                       )
tdf_l2

<p style = 'font-size:16px;font-family:Arial'>We can take another example where we want to find out the tickets that were resolved by a particular assignment group for example GSO L1 team and we also want to check all the reassignments that were done before the resolution.</p>

In [ ]:
npath_gso_l1 = NPath(data1 = tdf, 
                      data1_partition_column = ['sn_ticket'], 
                      data1_order_column = 'sn_audit_TS', 
                      mode = 'NONOVERLAPPING', 
                      symbols = ['fieldname = \'resolved_by\' and sn_ag IN (\'GSO L1 Cloud Support\',\'GSO L1 EU Restricted\') AND sn_state IN (\'Resolved\') AS resolved_gso_l1',
                                 'TRUE as OTHER'], 
                      pattern = 'OTHER?*.resolved_gso_l1', 
                      result = ['FIRST (sn_ticket of resolved_gso_l1) as sn_ticket', 
                                'FIRST (sn_audit_TS of resolved_gso_l1) as resolved_gso_l1_ts',
                                'COUNT (DISTINCT  sn_ag of ANY(other,resolved_gso_l1) ) as sn_ags', 
                                'ACCUMULATE (DISTINCT  cast(sn_ag as VARCHAR(20)) of ANY(other,resolved_gso_l1)) as path' 
                               ])

tdf_gso_l1=npath_gso_l1.result
tdf_gso_l1

<hr style="height:2px;border:none;">

<p style = 'font-size:20px;font-family:Arial'><b>4. Analysis and Visualization</b>
<p style = 'font-size:16px;font-family:Arial'> We can perform analysis on the data in-database and visualize the results by plotting the graphs and paths.</p>

In [ ]:
d1 = tdf_gso_l1.groupby(['path']).count()
d1= d1.assign(drop_columns = True,
              path = d1.path,
              ticket_count = d1.count_sn_ticket)
d1

In [ ]:
sn_ags_plot = tdf_gso_l1.groupby(['sn_ags']).count().sort(['count_sn_ticket'], ascending = False)
sn_ags_plot

In [ ]:
plot =  sn_ags_plot.plot(x=sn_ags_plot.sn_ags, y=sn_ags_plot.count_path,
                                 kind='bar',xlabel = 'Number of Reassignments', yabel = 'Count of tickets', 
                                 heading="Reassignments done before ticket resolved by GSO L1", figsize=(600, 400))
 
# Display the plot.
plot.show()

<hr style="height:1px;border:none;">
<p style = 'font-size:18px;font-family:Arial'><b>4.1  Sankey Charts</b></p>
<p style = 'font-size:16px;font-family:Arial'> In order to visualize the distribution of the different path of events, we typically use Sankey diagram of the aggregated over the paths reported by the NPATH command.


In [ ]:
from tdnpathviz.visualizations import plot_first_main_paths

In [ ]:
plot_first_main_paths(tdf_gso_l1,path_column='path',id_column='sn_ticket')

<hr style="height:2px;border:none;">
<p style = 'font-size:20px;font-family:Arial'><b>5. Json Export </b></p>
<p style = 'font-size:16px;font-family:Arial'>Once we got our patterns from the nPath function, we can also create a combined json table/file from that to be used in any of the BI tools for visualization if needed.<br>We will combine all our nPath results in a single dataframe to get all the kpis in one place.</p>

In [ ]:
tdf3=tdf.drop_duplicate(["sn_ticket","Site_id","sn_created_on","sn_created_on_DT"])
tdf3.shape

In [ ]:
out_json = tdf3.join(other = tdf_l2_l3,on = "sn_ticket", how = "left", rprefix ="l2_l3")\
               .join(other = tdf_l2,on = "sn_ticket", how = "left", rprefix ="l2")\
               .join(other = tdf_gso_l1,on = "sn_ticket", how = "left", rprefix ="gso_l1")
out_json  

In [ ]:
out_json_df = out_json.assign(drop_columns = True,
                              sn_ticket = out_json.sn_ticket,
                              site_id =out_json.Site_id,
                              sn_created_on_ts = out_json.sn_created_on,
                              sn_created_on_dt = out_json.sn_created_on_DT,
                              gso_l1_path = out_json.gso_l1_path,
                              gso_l1_week = out_json.resolved_gso_l1_ts,
                              l2_touched_path = out_json.l2_path,
                              l2_touched_week= out_json.l2_path_sn_audit_ts,
                              l2_cloudl3_bounces_path=out_json.path,
                              l2_cloudl3_bounces_week=out_json.path_sn_audit_week)
out_json_df

In [ ]:
copy_to_sql(out_json_df, table_name = 'out_json', if_exists = 'replace')

<p style = 'font-size:16px;font-family:Arial'>Next we will expand all our paths in an intermediate table so that we can get the kpi values as a single row per value.</p>

In [ ]:
query='''
create table demo_user.expanded_kpi_table as (
 select
    events_table.outkey as sn_ticket,
    out_df.site_id,
    out_df.sn_created_on_ts,
    out_df.sn_created_on_dt,
    'l2_touched' as kpi,
    events_table.tokennum,
    events_table.event,
    week_table.week

from (
    select
        events.outkey,
        events.tokennum,
        trim(regexp_replace(events.token, '\[|\]', '')) as event
    from table(
        strtok_split_to_table(DEMO_USER.out_json.sn_ticket, DEMO_USER.out_json.l2_touched_path, ',')
        returns(outkey VARCHAR(20), tokennum INTEGER, token VARCHAR(20))
        ) as events
    ) events_table

    inner join (
    select
        wk.outkey,
        wk.tokennum,
        trim(regexp_replace(wk.wk, '\[|\]', '')) as week
    from table(
        strtok_split_to_table(DEMO_USER.out_json.sn_ticket, DEMO_USER.out_json.l2_touched_week, ',')
        returns(outkey VARCHAR(20), tokennum INTEGER, wk VARCHAR(20))
        ) as wk
    ) week_table
    on week_table.outkey = events_table.outkey
    and week_table.tokennum = events_table.tokennum
    inner join DEMO_USER.out_json out_df
    on events_table.outkey = out_df.sn_ticket

UNION ALL

select
    events_table.outkey as sn_ticket,
    out_df.site_id,
    out_df.sn_created_on_ts,
    out_df.sn_created_on_dt,
    'gso_resolved_l1' as kpi,
    events_table.tokennum,
    events_table.event,
    week_table.week

from (
    select
        events.outkey,
        events.tokennum,
        trim(regexp_replace(events.token, '\[|\]', '')) as event
    from table(
        strtok_split_to_table(DEMO_USER.out_json.sn_ticket,DEMO_USER.out_json.gso_l1_path, ',')
        returns(outkey VARCHAR(20), tokennum INTEGER, token VARCHAR(20))
        ) as events
    ) events_table

left join (
    select
        sn_ticket,
        cast(cast(gso_l1_week as date) as varchar(20))week
    from DEMO_USER.out_json 
    ) week_table
    on week_table.sn_ticket = events_table.outkey
  
inner join DEMO_USER.out_json out_df
    on events_table.outkey = out_df.sn_ticket
     where events_table.tokennum is not null
    
UNION ALL

select
    events_table.outkey as sn_ticket,
    out_df.site_id,
    out_df.sn_created_on_ts,
    out_df.sn_created_on_dt,
    'l2_cloudl3_bounces' as kpi,
    events_table.tokennum,
    events_table.event,
    week_table.week

from (
    select
        events.outkey,
        events.tokennum,
        trim(regexp_replace(events.token, '\[|\]', '')) as event
    from table(
        strtok_split_to_table(DEMO_USER.out_json.sn_ticket,DEMO_USER.out_json.l2_cloudl3_bounces_path, ',')
        returns(outkey VARCHAR(20), tokennum INTEGER, token VARCHAR(20))
        ) as events
    ) events_table

   left join (
    select
        wk.outkey,
        wk.tokennum,
        trim(regexp_replace(wk.wk, '\[|\]', '')) as week
    from table(
        strtok_split_to_table(DEMO_USER.out_json.sn_ticket, DEMO_USER.out_json.l2_cloudl3_bounces_week, ',')
        returns(outkey VARCHAR(20), tokennum INTEGER, wk VARCHAR(20))
        ) as wk
    ) week_table
    on week_table.outkey = events_table.outkey
    and week_table.tokennum = events_table.tokennum
    inner join DEMO_USER.out_json out_df
    on events_table.outkey = out_df.sn_ticket
)
with data;
'''
execute_sql(query)

<p style = 'font-size:16px;font-family:Arial'>For output json, let us first create a table with json datatype.</p>

In [ ]:
query='''
create table demo_user.json_kpi_metrics 

(
    sn_ticket VARCHAR(20) CHARACTER SET LATIN NOT CASESPECIFIC,
    combined_kpis json(10000)
    )
primary index(sn_ticket)
;
'''
execute_sql(query)

<p style = 'font-size:16px;font-family:Arial'>Now we will insert the kpi values from the intermediate table in json format.</p>

In [ ]:
query='''
insert into demo_user.json_kpi_metrics 
select
    sn_ticket,
    json_compose (t2.sn_ticket,t2.site_id,
                t2.sn_created_on_ts,
                t2.sn_created_on_dt, t2.kpis)
from
    (
        select
            sn_ticket,
            site_id,
            sn_created_on_ts,
            sn_created_on_dt,
            json_agg(kpi,vals) as kpis
        from (
            select
                sn_ticket,
                site_id,
                sn_created_on_ts,
                sn_created_on_dt,
                kpi,
                json_agg(tokennum, event, week) as vals
            from demo_user.expanded_kpi_table
            group by 1,2,3,4,5
             ) as t1
        group by 1,2,3,4
    ) as t2;
'''
execute_sql(query)

In [ ]:
df = DataFrame('json_kpi_metrics')
df

In [ ]:
df.tdtypes

<p style = 'font-size:18px;font-family:Arial'><b>Conclusion</b></p>
<p style = 'font-size:16px;font-family:Arial'>Here in this notebook we have seen that with Teradata Vantage and ClearScape Analytics we can find patterns inside ServiceNow data. We can find common paths in our data and improve efficiency and quality of service we provide to ServiceNow customers. We have also seen how we can create a json from the data if we need an export in that format.</p>

<hr style="height:2px;border:none;">
<p style = 'font-size:20px;font-family:Arial'><b>6. Cleanup </b></p>
<p style = 'font-size:18px;font-family:Arial'><b>Work Tables</b></p>
<p style = 'font-size:16px;font-family:Arial'>We need to clean up our work tables to prevent errors next time.</p>

In [ ]:
db_drop_view('tickets_assignment_groups')

In [ ]:
tables = ['out_json', 'expanded_kpi_table', 'json_kpi_metrics']

# Loop through the list of tables and execute the drop table command for each table
for table in tables:
    try:
        db_drop_table(table_name = table)
    except:
        pass

<p style = 'font-size:18px;font-family:Arial'> <b>Databases and Tables </b></p>
<p style = 'font-size:16px;font-family:Arial'>We will use the following code to clean up tables and databases created for this demonstration.</p>

In [ ]:
%run -i ../run_procedure.py "call remove_data('DEMO_ServiceNow');"   # Takes about 10 seconds, optional if you want to use the data later

In [ ]:
remove_context()

<hr style="height:2px;border:none;">
<b style = 'font-size:18px;font-family:Arial'>Dataset Information</b>
<p style = 'font-size:16px;font-family:Arial'>Dataset used here is simulated data based on the actual ServiceNow audit data.</p>

<footer style="padding-bottom:35px; border-bottom:3px solid #91A0Ab">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            © 2025 Teradata. All rights reserved.
        </div>
    </div>
</footer>